# Data-mining of the Tweets
We have to collect data from July 13, 2009 to September 16, 2009 (announcement of elections, end of the drama). For the sake of monitoring the sentiment more safely, we will take +- 7 days to each date

In [1]:
el_announcement_2009 = "2009-07-06" # The date for the 2009 Japanese elections was first announced on July 13, 2009, by then Prime Minister Taro Aso. He decided to dissolve the House of Representatives and called for a general election.
el_elected_2009 = "2009-09-23" # The drama surrounding the Japanese elections of 2009 can be considered to have ended by September 16, 2009, when the newly elected Prime Minister Yukio Hatoyama was officially appointed by Emperor Akihito and his cabinet was announced. This marked the conclusion of the election process and the beginning of the new government.

In [2]:
from core.twitter import get_tweets_between_dates
import pandas as pd

# Loading the mapping file and keywords that we will be using to filter and collect the tweets.
mapping_df = pd.read_excel('./src/mappings/mapping_elections_jap.xlsx')
Party_JPN = mapping_df['Party_JPN'].to_list()
Chairman_JPN = mapping_df['Chairman_JPN'].to_list()
TAGS = ['総選挙'] + Party_JPN + Chairman_JPN

In [3]:
# Collecting maximum 3000 tweets per day. It will take a while to process (110 minutes).
# If 404 error, please use another VPN. It means that your IP has been banned from Twitter.
df = get_tweets_between_dates(TAGS, el_announcement_2009, el_elected_2009) 
df.to_excel('./src/data/2009_elections.xlsx', engine='xlsxwriter', encoding='utf-8')

🔎 Searching for 総選挙 from 2009-07-06 to 2009-07-07 (Attempt 1). Length: 0


Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=%E7%B7%8F%E9%81%B8%E6%8C%99+lang%3Aja+since%3A2009-07-06+until%3A2009-07-07+&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_correction

❌ ERROR: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=%E7%B7%8F%E9%81%B8%E6%8C%99+lang%3Aja+since%3A2009-07-06+until%3A2009-07-07+&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_corr

KeyboardInterrupt: 

# (Optional) Merging existing .xlsx data to get the one large dataset
This is purely for me, just because I've often wrongly collected data on different time periods, however this is still data that could be processed. Purpose of this notebook is to merge files, sort by datetime, drop duplicate tweets.

In [ ]:
import pandas as pd
from core.utils import convert_to_datetime

# This will take a while. It is not recommended to launch this on weak machines. It will require decent RAM and CPU power.
df_old = pd.read_excel('./src/data/raw/2009_elections_old.xlsx')
df_new = pd.read_excel('./src/data/raw/2009_elections_new.xlsx').drop(columns=['Unnamed: 0'])
old_columns = df_old.columns.to_list()
new_columns = df_old.columns.to_list()

print(f'Old columns: {len(old_columns)} | New columns: {len(new_columns)}')
print(f'Old: {len(df_old)} | New: {len(df_new)}')
# We will check if the columns are the same.
if old_columns == new_columns:
    print(f'✅ Columns are the same')

In [ ]:
# Just in case, we will convert the dates to datetime objects.
df_old = convert_to_datetime(df_old)
df_new = convert_to_datetime(df_new)
# count NaN values in tweet_url
print(f'Old: {df_old["tweet_url"].isna().sum()} | New: {df_new["tweet_url"].isna().sum()}')

In [ ]:
# Merge the two dataframes
df_merged = pd.concat([df_old, df_new])
print(f'Old: {len(df_old)} | New: {len(df_new)} | Merged: {len(df_merged)}')
# Sort by datetime
df_merged = df_merged.sort_values('tweet_date')

# Drop duplicate rows by tweet_id
df_merged = df_merged.drop_duplicates(subset=['tweet_conversationId', 'tweet_id', 'tweet_rawContent'])
print(f'Old: {len(df_old)} | New: {len(df_new)} | Merged: {len(df_merged)}')
df_merged.to_excel('./src/data/2009_elections.xlsx', index=False, encoding='utf-8', engine='xlsxwriter')